In [2]:
import pickle
import torch
from oa.dataset import ProcessedTS1x
from pathlib import Path
from torch.utils.data import DataLoader
import numpy as np

/home/xmcao/.conda/envs/install/envs/oa_reactdiff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
data =pickle.load(open('oa/data/transition1x/valid_addprop.pkl','rb'))
val_config = dict(
    datadir="oa/data/transition1x/",
    remove_h=False,
    bz=1,
    num_workers=0,
    clip_grad=True,
    gradient_clip_val=None,
    ema=False,
    ema_decay=0.999,
    swapping_react_prod=True,
    append_frag=False,
    use_by_ind=True,
    reflection=False,
    single_frag_only=False,
    only_ts=False,
    lr_schedule_type=None,
    lr_schedule_config=dict(
        gamma=0.8,
        step_size=100,
    ),  # step
)

# dataset
dataset = ProcessedTS1x(Path("oa/data/transition1x/valid_addprop.pkl"),
                        **val_config)

In [35]:
loader = DataLoader(
            dataset,
            4,
            shuffle=False,
            collate_fn=dataset.collate_fn,
        )

In [36]:
for batch in loader:
    break

In [37]:
batch

([{'size': tensor([7, 7, 8, 6]),
   'pos': tensor([[-0.7133,  1.0482,  0.2093],
           [ 0.4627,  0.8515, -0.3799],
           [ 0.8540, -0.3798, -0.3900],
           [-0.1908, -0.9900,  0.2587],
           [-1.1484, -0.1942,  0.6369],
           [ 0.9684,  1.7175, -0.7850],
           [-0.2326, -2.0533,  0.4501],
           [ 0.9710, -0.7751,  0.3282],
           [-0.2718, -0.6334,  0.7800],
           [-0.9579,  0.2788,  0.1746],
           [-0.0372,  0.7352, -0.7358],
           [ 1.1233,  0.1481, -0.6917],
           [-0.5841, -1.2778,  1.5905],
           [-0.2433,  1.5242, -1.4458],
           [ 1.7948,  0.5636, -0.6311],
           [ 0.5850,  0.1620, -0.2521],
           [-0.1701, -0.6257, -0.9703],
           [-1.3048, -0.7621, -0.1772],
           [-1.0856, -0.0382,  0.9139],
           [ 0.0571,  0.5641,  0.9443],
           [ 1.9916,  0.1592, -1.4894],
           [-1.8680, -0.0229,  1.6619],
           [-0.0608,  0.4873, -0.0240],
           [ 0.1199, -0.9621,  0.0478],


In [43]:
dataset[2145]

{'size_0': tensor(6),
 'pos_0': tensor([[ -9.9320, -12.5000,  -9.5595],
         [-10.0655, -11.4461,  -9.0829],
         [-10.2010, -10.3987,  -8.5865],
         [  9.9315,  12.4901,   9.5336],
         [ 10.0633,  11.4665,   9.0868],
         [ 10.2038,  10.3882,   8.6085]]),
 'one_hot_0': tensor([[0, 0, 0, 1, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 0, 1, 0]]),
 'charge_0': tensor([[8],
         [6],
         [8],
         [7],
         [7],
         [8]], dtype=torch.int32),
 'mask_0': tensor([0, 0, 0, 0, 0, 0]),
 'size_1': tensor(6),
 'pos_1': tensor([[-0.2595,  1.2641,  1.5652],
         [-0.3503,  0.6034,  0.5910],
         [-1.0986,  0.2937, -0.3527],
         [-0.1610, -0.9196, -1.3809],
         [ 0.8165, -0.9458, -0.7420],
         [ 1.0530, -0.2958,  0.3194]]),
 'one_hot_1': tensor([[0, 0, 0, 1, 0],
         [0, 1, 0, 0, 0],
         [0, 0, 0, 1, 0],
         [0, 0, 1, 0, 0],
         [0, 0, 